In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import datatable as dt
import numpy as np
# import mlb
import pickle as pkl
from tqdm import tqdm
from itertools import product
import lightgbm as lgb
from fuzzywuzzy import fuzz
import re
import numba as nb
from numba import njit
import xgboost as xgb
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/mlb-player-digital-engagement-forecasting/players.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/example_sample_submission.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/teams.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/seasons.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/example_test.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/train_updated.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/train.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/awards.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/mlb/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/mlb-player-digital-engagement-forecasting/mlb/__init__.py


In [2]:
""" Required modules inatlling """
#!pip install pip --upgrade
# !pip install MLB

' Required modules inatlling '

"""
Sample https://www.kaggle.com/code/brandenkmurray/mlb-predict-final/notebook
""""

** Data read started **

In [3]:
print("\n Reading all the required file started \n")
print("====="*20)
train = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/train_updated.csv")
teams = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/teams.csv")
players = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/players.csv")
awards_history = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/awards.csv")
print("====="*20)
print("\n Reading all the required file completed \n")


 Reading all the required file started 


 Reading all the required file completed 



In [4]:
train.head(5)

,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,20180101,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,20180102,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,20180103,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,20180104,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,20180105,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


In [5]:
test = True
if test:
    last_date = train['date'].max()
else:
    last_date = 20210430
print(" last date : {}".format(last_date))

 last date : 20210731


**SAMPLE DATA**

**EDA**

In [6]:
print("\n Shape of data \n")
print("==="*30)
print(train.shape)
print("==="*30)
print("\n NUmber of columns ")
print("==="*30)
print(train.columns)
print("==="*30)
print(train.info())
print("==="*30)


 Shape of data 

(1308, 12)

 NUmber of columns 
Index(['date', 'nextDayPlayerEngagement', 'games', 'rosters',
       'playerBoxScores', 'teamBoxScores', 'transactions', 'standings',
       'awards', 'events', 'playerTwitterFollowers', 'teamTwitterFollowers'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308 entries, 0 to 1307
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   date                     1308 non-null   int64 
 1   nextDayPlayerEngagement  1308 non-null   object
 2   games                    729 non-null    object
 3   rosters                  1307 non-null   object
 4   playerBoxScores          627 non-null    object
 5   teamBoxScores            627 non-null    object
 6   transactions             1194 non-null   object
 7   standings                623 non-null    object
 8   awards                   309 non-null    object
 9   events               

In [7]:
train.head(5)

,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,20180101,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,20180102,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,20180103,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,20180104,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,20180105,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


**Utility Functions**

In [8]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        print("\n column type column name {} and data type {}".format(col,col_type))
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            print("\n column min :{} col max : {} ".format(c_min,c_max))
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int64)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float64)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df



def unnest(data, name):
    try:
        date_nested_table = data[['date', name]]

        date_nested_table = (date_nested_table[
          ~pd.isna(date_nested_table[name])
          ].
          reset_index(drop = True)
          )

        daily_dfs_collection = []
        print("\n daily_dfs_collection started ")
        for date_index, date_row in date_nested_table.iterrows():
            daily_df = pd.read_json(date_row[name])

            daily_df['dailyDataDate'] = date_row['date']

            daily_dfs_collection = daily_dfs_collection + [daily_df]

        if daily_dfs_collection:
            # Concatenate all daily dfs into single df for each row
            unnested_table = (pd.concat(daily_dfs_collection,
              ignore_index = True).
              # Set and reset index to move 'dailyDataDate' to front of df
              set_index('dailyDataDate').
              reset_index()
              )
            return reduce_mem_usage(unnested_table, False)
        else:
            return pd.DataFrame()
    except Exception as e:
        print(e)
        print(f'unnest failed for {name}. returning empty dataframe')
        return pd.DataFrame()

**Derived Features**
> Create the features using function unnest
##################################################################################################
## Regex's for extracting and matching ejected player names and getting their playerId ##
##################################################################################################

In [9]:
eng = unnest(train, 'nextDayPlayerEngagement')
p_box_scores_og = unnest(train, 'playerBoxScores')
team_names = list(teams['teamName'].unique()) + ["Diamondbacks"]
team_regex = re.compile('|'.join(map(re.escape, team_names)))
team_full_names = list(teams['name'].unique()) + list(teams['teamName'].unique())
team_full_regex = re.compile('|'.join(map(re.escape, team_full_names)))


 daily_dfs_collection started 

 column type column name dailyDataDate and data type int64

 column min :20180101 col max : 20210731 

 column type column name engagementMetricsDate and data type object

 column type column name playerId and data type int64

 column min :112526 col max : 685503 

 column type column name target1 and data type float64

 column min :0.0 col max : 100.0 

 column type column name target2 and data type float64

 column min :0.0 col max : 100.0 

 column type column name target3 and data type float64

 column min :0.0 col max : 100.0 

 column type column name target4 and data type float64

 column min :0.0 col max : 100.0 

 daily_dfs_collection started 

 column type column name dailyDataDate and data type int64

 column min :20180329 col max : 20210731 

 column type column name home and data type int64

 column min :0 col max : 1 

 column type column name gamePk and data type int64

 column min :529406 col max : 635940 

 column type column name gameD

In [10]:
teams['teamName'].unique()

array(['Angels', 'D-backs', 'Orioles', 'Red Sox', 'Cubs', 'Reds',
       'Indians', 'Rockies', 'Tigers', 'Astros', 'Royals', 'Dodgers',
       'Nationals', 'Mets', 'Athletics', 'Pirates', 'Padres', 'Mariners',
       'Giants', 'Cardinals', 'Rays', 'Rangers', 'Blue Jays', 'Twins',
       'Phillies', 'Braves', 'White Sox', 'Marlins', 'Yankees', 'Brewers'],
      dtype=object)